In [3]:
from IPython.display import display
import folium

In [7]:
# Create a map centered at a specific location (latitude, longitude)
map_points = folium.Map(location=[52.08771347651753, 5.113090921625903], zoom_start=12)

# Add a marker to the map
folium.Marker(location=[52.08771347651753, 5.113090921625903], popup='Hello, World!').add_to(map_points)

# Display the map
display(map_points)


In [ ]:
# Save the map as an HTML file
map_points.save('../maps/Day1_Points.html')